Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 26-May-2025

This code uses the computational model by Shiu, PK, Sterne, GR, Spiller, N et al. (2024), Nature 634, 210–219, https://doi.org/10.1038/s41586-024-07763-9. 

In [3]:
from simulation_model import run_exp
from simulation_model import default_params as params
from brian2 import Hz
from brian2 import mV

import yaml
import pandas as pd 
import os

INFO       Cache size for target 'cython': 16873 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the 'C:\Users\Chris\.cython\brian_extensions' directory. [brian2]


In [5]:
#params

In [6]:
# w_syn is a free parameter related to the synaptic weight. It determines how much each synapse influences the postsynaptic membrane potential. 
# Scaling w_syn essentially results in scaling the "distance" between the resting potential and the firing threshold potential.
# Default: 0.275 * mV or 275.0 * uvolt
params['w_syn'] = 0.275 * mV
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 150. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr  : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

In [19]:
# Set experimental parameters
experiment_name = "BDN2"

# Connectome IDs in MANC
neurons_to_excite = [10093] #, 10107] # BDN2, Web
#neurons_to_excite = [32815, 31635, 32742, 31078, 10107] # DNg12 presynaptic to chief 9A in T1L 

freqs = [50,100,150]
path_to_completeness = "../data/simulation/completeness_manc_v1.csv"
path_to_connectivity = "../data/simulation/connectivity_manc_v1.parquet"

In [20]:
# Remove connectome IDs that are not in completeness.csv
df_completeness = pd.read_csv(path_to_completeness)
neurons_in_model = df_completeness['Unnamed: 0'].tolist()
neurons_to_excite = [(i) for i in neurons_to_excite if i in neurons_in_model]
print(len(neurons_to_excite))

1


In [21]:
# Create results folder if it does not exist
path_to_results = os.path.join("../data/simulation/", experiment_name)
if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

In [22]:
# Run simulation 
for f in freqs:
    # Change default stimulation frequency 
    params["r_poi"] = f * Hz

    run_exp(
        exp_name = str(f)+"Hz", #"{}_{}Hz".format(experiment_name, f), 
        neu_exc = neurons_to_excite,
        path_res = path_to_results,
        path_comp = path_to_completeness,
        path_con = path_to_connectivity,
        force_overwrite = True,
        params = params,
        n_proc = -1
    )

>>> Experiment:     50Hz
    Output file:    ..\data\simulation\BDN2\50Hz.parquet
    Exited neurons: 1
    Elapsed time:   16 s
>>> Experiment:     100Hz
    Output file:    ..\data\simulation\BDN2\100Hz.parquet
    Exited neurons: 1
    Elapsed time:   7 s
>>> Experiment:     150Hz
    Output file:    ..\data\simulation\BDN2\150Hz.parquet
    Exited neurons: 1
    Elapsed time:   9 s
